In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from tabulate import tabulate
from collections import Counter

In [2]:
dataset = ["I love playing football on the weekends",
 "I enjoy hiking and camping in the mountains",
 "I like to read books and watch movies",
 "I prefer playing video games over sports",
 "I love listening to music and going to concerts"]

In [3]:
def convert_to_lowercase(text):
    return text.lower()
dataset = [convert_to_lowercase(text) for text in dataset]
for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")

1: i love playing football on the weekends
2: i enjoy hiking and camping in the mountains
3: i like to read books and watch movies
4: i prefer playing video games over sports
5: i love listening to music and going to concerts


In [18]:
import re
import string

def remove_punctuation(text):
    if not isinstance(text, str):
        return text
    text = re.sub(r'[\n\r]+', ' ', text)
    punctuation_to_remove = string.punctuation.replace("'", "")
    text = re.sub(r"[{}]".format(re.escape(punctuation_to_remove)), " ", text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

dataset = [remove_punctuation(text) for text in dataset]

for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")

1: i love playing football on the weekends
2: i enjoy hiking and camping in the mountains
3: i like to read books and watch movies
4: i prefer playing video games over sports
5: i love listening to music and going to concerts


In [26]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    if isinstance(text, str):
        return ' '.join(word for word in text.split() if word.lower() not in stop_words)
    return text
stop_words = set(stopwords.words('english'))
# Apply all transformations
dataset = [remove_stopwords(text) for text in dataset]

# Print the cleaned dataset
for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")

1: love playing football weekends
2: enjoy hiking camping mountains
3: like read books watch movies
4: prefer playing video games sports
5: love listening music going concerts


In [28]:
def remove_suffixes(text):
    if isinstance(text, str):
        words = text.split()
        stripped_words = []
        for word in words:
            # Basic handling for common suffixes
            if word.endswith("ing") and len(word) > 4:
                stripped_words.append(word[:-3])
            elif word.endswith("es") and len(word) > 3:
                stripped_words.append(word[:-2])
            else:
                stripped_words.append(word)
        return ' '.join(stripped_words)
    return text

# Apply the new step AFTER the previous processing
dataset = [remove_suffixes(text) for text in dataset]

# Print the final output
for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")

1: love play football weekends
2: enjoy hik camp mountains
3: like read books watch movi
4: prefer play video gam sports
5: love listen music go concerts


In [46]:
!pip install autocorrect

from autocorrect import Speller

spell = Speller(lang='en')

def autocorrect_text(text):
    if isinstance(text, str):
        return ' '.join(spell(word) for word in text.split())
    return text

# Apply autocorrect
dataset = [autocorrect_text(text) for text in dataset]

# Print results
print("\nAutocorrected dataset:")
for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")


Autocorrected dataset:
1: love play football weekends
2: enjoy his camp mountains
3: like read books watch move
4: prefer play video gam sports
5: love listen music go concerts


In [48]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset)

In [50]:
k = 2 # Define the number of clusters
km = KMeans(n_clusters=k)
km.fit(X)
# Predict the clusters for each document
y_pred = km.predict(X)
# Display the document and its predicted cluster in a table
table_data = [["Document", "Predicted Cluster"]]
table_data.extend([[doc, cluster] for doc, cluster in zip(dataset, y_pred)])
print(tabulate(table_data, headers="firstrow"))

Document                         Predicted Cluster
-----------------------------  -------------------
love play football weekends                      0
enjoy his camp mountains                         1
like read books watch move                       0
prefer play video gam sports                     0
love listen music go concerts                    0


In [52]:
# Print top terms per cluster
print("\nTop terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(k):
 print("Cluster %d:" % i)
 for ind in order_centroids[i, :10]:
     print(' %s' % terms[ind])
 print()


Top terms per cluster:
Cluster 0:
 love
 play
 football
 weekends
 go
 sports
 music
 concerts
 video
 gam

Cluster 1:
 camp
 enjoy
 his
 mountains
 weekends
 listen
 concerts
 football
 gam
 go



In [54]:
# Calculate purity
total_samples = len(y_pred)
cluster_label_counts = [Counter(y_pred)]
purity = sum(max(cluster.values()) for cluster in cluster_label_counts) / total_samples
print("Purity:", purity)

Purity: 0.8
